# Init

In [2]:
import pandas as pd

# Data

In [4]:
# Replace 'your_file.csv' with the path to your CSV file
csv_data = pd.read_csv('edge_distances_osrm.csv')

# Display the first few rows of the CSV data
csv_data.head()

,edge_name,distance_m,duration_s,from_node,to_node,distance_km,duration_min,avg_speed_kmh,u_ij,a_ij
0,E18 North,9333.6,628.5,Täby,Bergshamra,9.3336,10.475000,53.462148,2000,3.182180
1,E18 South,8989.9,704.5,Bergshamra,T-Centralen,8.9899,11.741667,45.938453,2000,2.838893
2,E20 East,8025.1,857.8,Lidingö,T-Centralen,8.0251,14.296667,33.679599,2000,2.331546
3,279 West,15364.8,1137.2,Vällingby,Bergshamra,15.3648,18.953333,48.639887,1200,1.055223
4,275 West,13454.3,1302.3,Vällingby,T-Centralen,13.4543,21.705000,37.192260,1200,0.921447


### Demand split

In [6]:
# Chat Gpt generated
# Construct the DataFrame
data = {
    "Node": [
        "Farsta Shopping Centre", "Skärholmen", "Vällingby", "Täby", "Lidingö",
        "Nacka Forum", "Sollentuna", "Bergshamra", "Avicii Arena", "Årstaberg", "T-Centralen"
    ],
    "Approx. Population": [
        60_523, 33_662, 71_042, 77_744, 48_162, 110_633, 76_237, 7_300, 70_779, 12_486, 40_312
    ],
    "Source": [
        "Farsta borough (2022 estimate)",
        "Skärholmen borough (2004)",
        "Hässelby-Vällingby borough (2014)",
        "Täby Municipality (31 Dec 2024)",
        "Lidingö urban area (2023)",
        "Nacka Municipality (2023)",
        "Sollentuna Municipality (2022)",
        "Bergshamra suburb (approx.)",
        "Östermalm borough (2014)",
        "Årsta locality (recent)",
        "Norrmalm district"
    ]
}

_df = pd.DataFrame(data)


In [8]:
import itertools

# Create a new DataFrame with demand for each Node from all other Nodes
demand_data = []

data_split = 0.9

for source, target in itertools.product(_df["Node"], repeat=2):
    if source != target:  # Exclude self-demand
        source_population = _df.loc[_df["Node"] == source, "Approx. Population"].values[0]
        if target == "T-Centralen":
            demand = int(source_population * data_split)
        else:
            demand = int(source_population * (1 - data_split) / (_df.shape[0] - 2))  # Exclude T-Centralen and itself
        demand_data.append({
            "Source": source,
            "Target": target,
            "Demand": demand
        })

demand_df = pd.DataFrame(demand_data)
demand_df.head()

,Source,Target,Demand
0,Farsta Shopping Centre,Skärholmen,672
1,Farsta Shopping Centre,Vällingby,672
2,Farsta Shopping Centre,Täby,672
3,Farsta Shopping Centre,Lidingö,672
4,Farsta Shopping Centre,Nacka Forum,672


# Modeling